## HotChili Analytics 

In [34]:
%matplotlib inline
%load_ext zipline

The zipline extension is already loaded. To reload it, use:
  %reload_ext zipline


In [35]:
import pandas as pd
#pd.set_option("max_colwidth", 400)
#pd.set_option("display.max_rows", 100000)
#pd.set_option("display.max_columns", 1000)
#pd.set_option('precision', 2)
#pd.options.display.float_format = '{:20,.2f}'.format

## Zipline Backtest: Jupyter  Magic Cell (%%) Execution of zipline code in cell containing command line command

In [36]:
%%zipline --start=2018-1-1 --end=2020-8-10 -b sharadar-eqfd -o /home/hca-blog/hca/hca-live-rel/nb/HCA_Trendfollowing.pkl

# Source: adapted from various algos on quantopian
# HCA Conversion Date: 08-14-2020
# Conversion Author: Anthony garner

# Simple trend following portfolio
import matplotlib.pyplot as plt
import pandas as pd
import logging as log

from zipline.api import order, cancel_order, get_open_orders, symbol, symbols, date_rules, time_rules, order_target_percent, record, schedule_function, get_datetime
from trading_calendars import get_calendar

def initialize(context):
    schedule_function(func=trade, date_rule=date_rules.every_day(),
                      time_rule=time_rules.market_open(),half_days=True)
    schedule_function(func=cancel,time_rule=time_rules.market_close(minutes=5),  
                      date_rule=date_rules.every_day(),half_days=True)  
    schedule_function(func=reorder, time_rule=time_rules.market_open(minutes=5),  
                      date_rule=date_rules.every_day(),half_days=True)
    context.asserts = symbols('SPY')
    context.bonds = symbol('SHY') 
    context.rebalance_date = 0
    context.fired = False
    context.rebalance_inteval = 'M'#'Q', #'D', #'M' #'Q' #'Y'
    context.top_n_by_momentum = pd.Series()  
    #Choose X stocks out of portfolio of Y stocks- how many stocks to hold - top X by momentum 
    context.stocks=1
    #Lookback for momentum calculation
    context.momentum_days=60
    #set at less than 1 to ensure no leverage
    context.leverage_buffer=0.99
    #Set to 0 to reject any stocks with negative momentum, set to -1 to accept stocks with negative momentum
    context.trend =0.0
    context.reorder_dict = {}  


    
def handle_data(context, data):
    record(SPY=data[symbol('SPY')].price)

def is_new_day(context, now):
    return ( (now.year > context.rebalance_date.year) or (now.month > context.rebalance_date.month) or((now.month == context.rebalance_date.month) and (now.day > context.rebalance_date.day)))             
    
def is_new_month(context, now):
    return ((now.year > context.rebalance_date.year) or ((now.year == context.rebalance_date.year) and (now.month > context.rebalance_date.month)))

def is_new_quarter(context, now):
    return ((now.year > context.rebalance_date.year) or ((now.year == context.rebalance_date.year) and (now.month == context.rebalance_date.month + 3)))
    
def is_new_year(context, now):
    return (now.year > context.rebalance_date.year)

def need_rebalance(context, now):
    return ((context.rebalance_inteval == 'Y' and is_new_year(context, now))or 
           (context.rebalance_inteval == 'Q' and is_new_quarter(context, now)) or 
           (context.rebalance_inteval == 'M' and is_new_month(context, now)) or 
           (context.rebalance_inteval == 'D' and is_new_day(context, now)))


    # Compute momentum
def compute_momentum(context,data):  
    price_history = data.history(context.asserts, "price", context.momentum_days+5, "1d")
    momentum = price_history.pct_change(context.momentum_days).iloc[-1]
    #for index,value in momentum.items():
        #log.debug("unfiltered momentun"+" "+ str(index)+" "+ str(value) )
    context.top_n_by_momentum = momentum.nlargest(context.stocks).where(momentum>context.trend).dropna()
    #for index,value in context.top_n_by_momentum.items():
        #log.debug("context.top_n_by_momentun"+" "+str(index)+" " + str(value) )
    return context.top_n_by_momentum

def init_portfolio(context, data):
    weights=0.0
    reserve_allocation=0.0
    compute_momentum(context, data)
    for index,value in context.top_n_by_momentum.items():
        if data.can_trade(index):
            weights =weights +1/context.stocks
            order_target_percent(index, (1/context.stocks)* context.leverage_buffer)
    #Assign weighting and an order to the reserve asset if and when appropriate
    if weights <1 and data.can_trade(context.bonds):
        reserve_allocation=1-weights
        order_target_percent(context.bonds, reserve_allocation* context.leverage_buffer)     
        
def rebalance(context, data):
    weights=0.0
    reserve_allocation=0.0
    compute_momentum(context, data)
    for x in context.portfolio.positions:
        if (x not in context.top_n_by_momentum and x != context.bonds):
            order_target_percent(x, 0)            
    
    for index,value in context.top_n_by_momentum.items():
        if data.can_trade(index)and index != context.bonds:
            weights =weights +1/context.stocks
            order_target_percent(index, (1/context.stocks)*context.leverage_buffer)
        elif data.can_trade(index) and index != context.bonds: 
            order_target_percent(index, 0)
    #Assign weighting and an order to the reserve asset if and when appropriate        
    if data.can_trade(context.bonds):
        reserve_allocation=1-weights
        order_target_percent(context.bonds, reserve_allocation* context.leverage_buffer)        

#Will be called daily. 
def trade(context, data):
    if not context.fired:
        context.rebalance_date = get_datetime()
        log.info("build portfolio at " + str(context.rebalance_date))
        init_portfolio(context, data)
        context.fired = True
        now = get_datetime()
    else:
        now = get_datetime()
        if (need_rebalance(context, now)):
            log.info("new rebalance arrivied:" + str(now))
            rebalance(context, data)
            context.rebalance_date = now
    #open_orders = get_all_open_orders()  
    #for order in open_orders:  
                #log.info("Rebalance Order {0:s} for {1:,d} shares" 
            #.format(order.sid.symbol,order.amount))  
#Called Daily to replace/re-order partially or unfilled orders
def cancel(context, data):  
    open_orders = get_all_open_orders()  
    for order in open_orders:  
        log.info("X CANCELED {0:s} with {1:,d} / {2:,d} filled" 
            .format(order.sid.symbol,  
                    order.filled,  
                    order.amount))  
        cancel_order(order)  
        context.reorder_dict[order.sid] = order.amount - order.filled

def get_all_open_orders():  
    from itertools import chain  
    orders = chain.from_iterable(get_open_orders().values())  
    return list(orders)  
#Called Daily to replace/re-order partially or unfilled orders
def reorder(context, data):  
    for stock, amount in context.reorder_dict.items():  
        order(stock, amount)  
        log.info("Reorder {stock} {amount}".format(stock=stock, amount=amount))  
    context.reorder_dict = {}
    
def analyze(context, perf):
    ax1 = plt.subplot(211)
    perf.portfolio_value.plot(ax=ax1)
    ax2 = plt.subplot(212, sharex=ax1)
    perf.SPY.plot(ax=ax2)
    plt.gcf().set_size_inches(18, 8)
    plt.show()

## Run back test via Command Line

In [1]:
#! zipline run -f /home/hca-blog/hca/hca-live-rel/strat/HCA_Trendfollowing.py --start 2020-2-28 --end 2020-8-10 -b sharadar-eqfd -o /home/hca-blog/hca/hca-live-rel/nb/HCA_Trendfollowing.pkl

/home/hca-blog/anaconda3/envs/env_hca/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/home/hca-blog/anaconda3/envs/env_hca/lib/python3.6/site-packages/odo/backends/pandas.py:94: FutureWarning: pandas.tslib is deprecated and will be removed in a future version.
You can access NaTType as type(pandas.NaT)
  @convert.register((pd.Timestamp, pd.Timedelta), (pd.tslib.NaTType, type(None)))
[2020-09-05 13:32:52.291499] WARNING: Loader: Refusing to download new benchmark data because a download succeeded at 2020-09-05 13:01:46.365526+00:00.
[2020-09-05 13:32:52.322432] WARNING: Loader: Refusing to download new treasury data because a download succeeded at 2020-09-05 13:01:48.545510+00:00.
build portfolio at 2020-02-28 21:00:00+00:00
[2020-09-05 13:32:53

## Ingest Sharadar Funds Assets for today.

In [38]:
# Ingest Sharadar Funds Assets for today.
# Only need to ingest Funds for this algo, and this takes less processing time and system memory than ingesting
#  all of Sharadar Equities plus Funds bundle (sharadar-eqfd)
#!date
#!zipline ingest -b sharadar-funds

## Command Line Execution of zipline with live trading
###    Note: IB-TWS or IB-Gateway must be running, with broker-acct and broker-uri port being correct to live trade.

In [39]:
#!zipline  run -s 08-12-2020 -f /home/hca-blog/hca/hca-live-rel/strat/HCA_Trendfollowing_live.py --broker ib --broker-uri 127.0.0.1:7499:1301 --broker-acct DU1568488 --bundle sharadar-eqfd --data-frequency daily  --state-file  /home/hca-blog/hca/hca-live-rel/strat/strategy.state --realtime-bar-target /home/hca-blog/hca/hca-live-rel/nb/logs/realtime-bars/ 


## Command Line Ingestion of sharadar-prices bundle

In [41]:
### !zipline ingest -b sharadar-prices

In [42]:
### !ls -l ~/.zipline/data/sharadar-eqfd

In [43]:
### !zipline bundles